In [1]:
import pandas as pd
import numpy as np
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Continuous Bag of Words (CBOW)
Or, how to predict a word given its context

### Notation

- $w$ - word
- $x$ - input vector
- $x_k$ - non-zero element of input vector
- $V$ - numbers of words in vocabulary
- $W$ - input weight matrix
- $W^{\prime}$ - output weight matrix
- $h$ - hidden layer vector
- $N$ - number of neurons in the hidden layer
- $y$ - output vector
- $C$ - number of context words


### Layers

- Input layer ($x$) - vector  - $V\times1$ one-hot input vector of context word
- Hidden Layer ($h$) - vector - $N \times 1$ 
- Output Layer ($y$) - vector - $V\times1$ output vector of predicted word whose values sum to 1

## Single word context

i.e., $C=1$

![](images/CBOW_viz-01.png)

![](images/CBOW_viz-02.png)

One-hot encoding of $x$ results in only a single column of the matrix multiplication ($W^T \times x$) output having non-zero values.  This column of $W^T$ corresponds to the $k^{th}$ row of $W$.  Since the hidden layer is the result of this multiplication:

$$h = {W_{k,:}}^T := v_c$$ 

$v_c$ is just the variable name which corresponds to the context vector ${W_{k,:}}^T$

![](images/CBOW_viz-03.png)

The hidden layer $h$ is then multiplied by the output weight matrix $(W^{\prime T}\times h)$ to get the linear form of the output vector $u$.  Each row of that column vector corresponds to a single word in the vocabulary. The corresponding column in the output weight matrix, $W^{\prime}_{:,i}$, is referred to as the word vector $v_{w_i}$. Note the difference between $w$, which refers to a word in vocabulary, and $W$ as well as $W^{\prime}$, which refer to the weight matrices.

$u$ is the linear component of the output vector, formed by the product of the word and context vectors, ${v_{w_i}}^T v_c$.  In order to convert the linear output into a probability, for which the sum of the vector is $1$, A non-linear transformation is applied (softmax).  Each value in the output vector $y$, corresponds to the probability of the that word being the predicted word given the context. 

$$p(w_i\mid c) = \dfrac{\exp{u_i}}{\sum\limits_{j=1}^V\exp{u_j}} = \dfrac{\exp {v_{w_i}}^Tv_c}{\sum\limits_{j=1}^V\exp {v_{w_j}}^Tv_c}$$


## Multiple word context

i.e., $C>1$

![](images/CBOW_viz-04.png)

![](images/CBOW_viz-05.png)

In multiple word context CBOW there are multiple one-hot input vectors (one for each context word).  There is still only a single input weight matrix, $W$, which is matrix mutliplied by the average input vector, i.e., $W^T \times \frac{1}{C}\sum\limits_{i=1}^Cx^i$.  Since this is the summation of multiple one hot vector, instead of there being only a single value from each column of the transposed weight matrix, there is one value for each input vector.  Put another way, for a context of $C$ words, each context vector has only a single value of 1 and the rest of their values are zero.  I can represent the index of that singular 1 value as $x^1_k$ for the first word, $x^2_k$ for the second word, and so on, through $x^C_k$.  Since their values are 1, I only care about the list of indices.  I then find the corresponding column indices in the input weight matrix, and take the average, which gives me the value for the hidden layer of that neuron. 

The rest of the CBOW model continues the same as it did for the single word context CBOW explanation.

## Loss Functions

In order to learn the correct values for the weight matrices you need to use backpropagation, which requires a loss function to guide it. There are multiple ways of calculating loss functions for this formulation, presented below are the most commonly used ones. 

### Cross Entropy

Cross entropy comes from information theory and describes the average number of bits required to identify an event (or in our case a word), if we use a coding scheme optimized for an "unnatural" distribution, rather than the true distribution.  The more different the two distrubtions are, the more bits required to describe the event.  In terms of a loss function, the cross entropy is a measure of how different the model distribution is from the true distribution, based on the average number of bits the model needs to describe an event drawn from the true distribution.  When the cross entropy is zero, then the model distribution perfectly models the true distribution.  This is why we can use cross entropy as a loss function and why we want to minimize the cross entropy.  

For discrete distributions the formulation for cross entropy is:

$$ H(p,q) = -\sum\limits_{x}p(x)\log q(x)$$

Where $p$ is the true distribution, $y$, and $q$ is the model distribution, $p(w_i \mid c)$.  

$$\mathcal{L}_{\theta} = -\sum\limits_{i=1}^Vy_i\log p(w_i\mid c)$$

Since in the true distribution there is only one correct word for the context, $y$ is one hot encoded and $y_i=0$ for all values of $i$ **except** when $w_i$ is the output word, denoted by $w_o$. 

\begin{align}
\mathcal{L}_{\theta}  &= -\sum\limits_{i=1}^Vy_i\log p(w_i\mid c) \\
&=-\log p(w_o\mid c) \\
&= -log\dfrac{\exp{u_o}}{\sum\limits_{j=1}^V\exp{u_j}} \\
&= -u_o + \log \sum\limits_{j=1}^V\exp\left(u_j\right) \\
&= -{v_{w_o}}^Tv_c + \log \sum\limits_{j=1}^V\exp\left( {v_{w_j}}^Tv_c\right) := E
\end{align}

Given this likelihood function, in order to get the correct weight matrices (or, as done in this case, their corresponding vectors) starting from random values we need to use gradient descent, which requires solving for the gradient of the loss function with respect to the two weight matrices.  Starting with the output weight matrix.

\begin{align}
\dfrac{\partial E}{\partial v_{w_i}} &= \dfrac{\partial E}{\partial u_i}\dfrac{\partial u_i}{\partial v_{w_i}}\\
&=\left(-\dfrac{\partial u_o}{\partial u_i} +\dfrac{\dfrac{\partial \sum\limits_{j=1}^V\exp\left(u_j\right)}{\partial u_i}}{\sum\limits_{j=1}^V\exp\left(u_j\right)}\right)\left(\dfrac{\partial{v_{w_i}}^Tv_c }{\partial v_{w_i}}\right) \\
&=\left(-\delta_i +\dfrac{\frac{\partial \exp(u_1)}{\partial u_i} + \frac{\partial \exp(u_1)}{\partial u_i} + \ldots + \frac{\partial \exp(u_i)}{\partial u_i} + \ldots + \frac{\partial \exp(u_V)}{\partial u_i}}{{\sum\limits_{j=1}^V\exp\left(u_j\right)}}\right)v_c \\
&= \left(-\delta_i + \dfrac{\exp(u_i)\dfrac{\partial}{\partial u_i}u_i}{{\sum\limits_{j=1}^V\exp\left(u_j\right)}}\right)h_i \\
&= \left(-\delta_i + p(w_i\mid c) \right)h_i \\
&= \left(y_i - \delta_i\right) h_i
\end{align}

Where $\delta_i$ is the delta function, i.e., it has a value of $1$ when $i^{th}$ is the acutal output word, $w_o$, otherwise its value is $0$.  Plugging this into the gradient descent equation:

$$ {v_{w_i}}^{new} = {v_{w_i}}^{old} - \eta \left(y_i - \delta_i\right) h_i $$

or, alternatively:

$$ {W^{\prime}_{:,i}}^{new} = {W^{\prime}_{:,i}}^{old} - \eta \left(y_i - \delta_i\right) h_i $$

Now to derive the update equation for the input weight matrix.

\begin{align}
\dfrac{\partial E}{\partial W^i_{k,n}} &= \sum\limits_{j=1}^V\dfrac{\partial E}{\partial u_j}\dfrac{\partial u_j}{\partial h_n}\dfrac{\partial h_n}{\partial W^i_{k,n}}\\
&= \sum\limits_{j=1}^V \left(y_j - \delta_j\right)\left({v_{w_j}}^T\right)\left( \dfrac{\partial}{\partial W^i_{k,n}} \dfrac{1}{C}\sum\limits_{i=1}^C{W^i_{k,n}}\right)\\
&= \sum\limits_{j=1}^V \left(y_j - \delta_j\right)\left({v_{w_j}}^T\right)\left(1\right)\\
\end{align}



## Implementation of single context CBOW

In [2]:
# One hot encoding
def onehot(sentence):
    sent_clean = re.sub("[^a-zA-Z\s\']+", "", sentence)
    words = sent_clean.lower().split(" ")
    words_unq = list(set(words))
    words_df = pd.DataFrame(np.identity(len(words_unq)),columns = words_unq)
    return words_df

In [ ]:
sent = "It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the season of Darkness, it was the spring of hope, it was the winter of despair, we had everything before us, we had nothing before us, we were all going direct to Heaven, we were all going direct the other way"
words_df = onehot(sent)
words_df.head()

Let's choose a hidden layer of 3 neurons (although you can choose one of any size)<br>

For the weight matrices you want to initiliaze them to some small random values

In [ ]:
V = len(words_df)
N = 3
Wi = np.random.rand(V,N) - 0.5
Wo = np.random.rand(N,V) - 0.5

Let's predict the next word given the previous word "worst"

In [ ]:
def predict_one_context(df,word,Wi,Wo):
    h = np.dot(np.transpose(Wi),words_df[word].as_matrix())
    y_act = np.dot(np.transpose(Wo),h)
    y_act_full = np.dot(np.transpose(Wo),np.transpose(Wi))
    y_num = np.exp(y_act)
    y_denom = np.sum(np.exp(y_act_full),1)
    return y_num/y_denom

In [ ]:
word = "worst"
word_pred = "of"
y_i = predict_one_context(words_df,word,Wi,Wo)
y_i[words_df.columns.get_loc(word_pred)]

The initial weight matrices are generated at random.  To learn the proper weight, you need to to maximize the probability likelihood, using the word and context vectors as the paramters, i.e. $\theta = \{{v_{w_i}},v_c\}$.  However, since we are working with a loss function we can minimize the negative likelihood and use the log likelihood to make things easier.

\begin{align}
\ell\{\theta\} &= -\log p(w_i\mid c;\theta) \\
&= -\log \prod\limits\\
\end{align}

$$L(\theta) = \prod\limits_{i=1}^Vp(w_i\mid c;\theta) = \prod\limits_{i=1}^V \dfrac{\exp {v_{w_i}}^Tv_c}{\sum\limits_{j=1}^V\exp {v_{w_j}}^Tv_c}$$

taking the log

$$\ell(\theta)  = \sum\limits_{i=1}^V {v_{w_i}}^Tv_c - \sum\limits_{i=1}^V \sum\limits_{j=1}^V\exp {v_{w_j}}^Tv_c$$